In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
import torch.nn as nn
import numpy as np

from modules.models_util import EfficientNet, ViT, VGG19, Erase
from modules.training_util import PytorchDataset

torch.manual_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [6]:
weights_path = [
    'notebooks/vit/models/01-31-2024 15-18-02/epoch_12.pth',            #
    'notebooks/vit/models/2024-02-03 21-45-07/best.pth',                #
    'notebooks/vit/models/2024-02-03 21-53-43/best.pth',                #
    'notebooks/vgg19/models/2024-02-04 16-46-29/epoch_15.pth',          #
    'notebooks/efficientnet/models/2024-02-05 16-41-35/epoch_15.pth',   #
    'notebooks/efficientnet/models/2024-02-06 17-36-09/best.pth',
    'notebooks/vit/models/2024-02-06 19-21-50/best.pth',
    'notebooks/vit/models/2024-02-11 08-28-01/epoch_8.pth',
    'notebooks/vit/models/2024-02-11 07-01-38/epoch_20.pth',            # sgkf-8-1-1-size-540
    'notebooks/vit/models/2024-02-23 12-16-05/epoch_14.pth'              # balanced_train 
    
]
# model = EfficientNet().load(weights_path[-1].replace('\\','/'))
model = ViT(architecture='b32').load(weights_path[-1].replace('\\','/'), fully_trainable=False)
model.heads = nn.Sequential()
# model = VGG19.load(weights_path[8].replace('\\','/'))
model.to(device)
model.eval()

# for param in model.parameters():
#     param.requires_grad = False

Weights for model VisionTransformer loaded from notebooks/vit/models/2024-02-23 12-16-05/epoch_14.pth


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [7]:
base_path = 'E:/Datasets/cor-splits'
# dataset_dir = os.path.join(base_path,'sgkf-8-1-1-size-540')
dataset_dir = os.path.join(base_path,'balanced_train')

data_transforms = ViT().data_transforms
dataset = PytorchDataset(dataset_dir, data_transforms, batch_size=16, training_phases=['test'], num_workers=2)
dataloader = dataset.dataloaders['test']

In [8]:
features = []
features_labels = []
with torch.no_grad():
    for i, (inputs, labels, paths) in enumerate(dataloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        bs, seq_length, _, _ = inputs.size()
        # inputs_flatten = inputs.view(bs, seq_length, -1)
        # outputs = transformer_backbone(inputs)
        outputs = model(inputs)
        
        features.extend(outputs)
        features_labels.extend(labels)
        
        _, predicted = torch.max(outputs, 1)

c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torch\nn\modules\activation.py:1196: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  return torch._native_multi_head_attention(


In [9]:
features_list = [feat.cpu().numpy() for feat in features]
features_array = np.array(features_list)
len(features_array)

1192

In [10]:
labels_list = [label.cpu().numpy() for label in features_labels]
labels_array = np.array(labels_list)
len(labels_array)

1192

In [11]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

In [12]:
svm_classifier = svm.SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [13]:
y_pred = svm_classifier.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print("SVM accuracy:", accuracy)

SVM accuracy: 0.9707112970711297
